In [1]:
import deorbit
import matplotlib.pyplot as plt
import deorbit.data_models
from deorbit.predictor import EKF, EKFOnline
from deorbit.utils.dataio import load_sim_data, load_sim_config
import numpy as np
from deorbit.observer import Observer


In [2]:
save_path = "eg/EKF_example_noise/"

sim_data = load_sim_data(save_path)
sim_config = load_sim_config(save_path)

if sim_data is None or sim_config is None:
    sim = deorbit.simulator.run(
        "adams_bashforth",
        "coesa_atmos_fast",
        initial_state=np.array((deorbit.constants.EARTH_RADIUS + 150000, 0, 0, 7820)),
        noise_types = {"gaussian": {"noise_strength": 0.01}, "impulse": {"impulse_strength": 0.1, "impulse_probability": 1e-5}},
        time_step=0.1,
    )
    sim_data = sim.gather_data()
    sim_config = sim.export_config()
    sim.save_data(save_path)
else:
    print("Loaded data from file")

Loaded data from file


In [3]:
obs = Observer(number_of_radars=3)
sim_states = sim_data.state_array()
sim_times = sim_data.times

obs.run(sim_states=sim_states, sim_times=sim_times, checking_interval=100)

observation_times = obs.observed_times
observation_states = obs.observed_states

In [4]:
# Define process and measurement noise covariance matrices, think this noise should be alot bigger
Q = np.diag([0.1, 0.1, 0.01, 0.01])#process noise
R = np.diag([1, 1, 0.1, 0.1]) #measurement noise
P = np.diag([1, 1, 1, 1]) #error covariance matrix - represents uncertainty in state estimation

# Measurement matrix H (assuming all states are measured directly??????) -- for now
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

dt = sim_config.simulation_method_kwargs.time_step

ekf = EKF()

In [6]:
# Initialize at a later time so velocity != 0
start_step = 10
ekf_online = EKFOnline(ekf, sim_states[start_step], sim_times[start_step], P)

In [7]:
# A generator that yields the next observation and the time it was observed. This is a stand in for a real observer.
def ekf_generator(sim_times, observation_states, observation_times, obs_uncertainties):
    if obs_uncertainties.ndim == 2:
        R = obs_uncertainties
    dt = sim_times[1] - sim_times[0]
    for time in sim_times:
        j = np.argmax(observation_times > time - dt / 2)
        if np.abs(observation_times[j] - time) < dt / 2:
            if obs_uncertainties.ndim == 3:
                R = obs_uncertainties[j]
            yield time, observation_states[j], R
        else:
            yield time, None, None

In [8]:
ekf_generator_instance = ekf_generator(sim_times, observation_states, observation_times, P)
# Skip ahead to start
[next(ekf_generator_instance) for _ in range(start_step-1)]
for time, observation, R in ekf_generator_instance:
    ekf_online.step(time, Q, observation, R, H)

KeyboardInterrupt: 